# Data Mining Project - zillow

Data Loading

In [10]:
# imports here:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load datasets here:
train_data = pd.read_csv('../input/train_2016_v2.csv')
properties_data = pd.read_csv('/Users/albertasuncion_mac/Documents/SMU/7331 - Data Mining/GitHub/7331 - Data Mining/data_mining_group_project/input/properties_2016.csv')
df = properties_data
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


# Business Understanding
<b>10 points</b>

<i>
<b>Description:</b><br/>
Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). Describe how you would define and measure the outcomes from the dataset. That is, why is this data important and how do you know if you have mined useful knowledge from the dataset? How would you measure the effectiveness of a good prediction algorithm? Be specific.
</i>

<b>Answer:</b><br/>
...

Home ownership is the american dream and, for most, the single largest purchase and investment of their lifetime. 

# Data Meaning Type
<b>10 points</b>

<i>
<b>Description:</b><br/>
Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file.
</i>

<b>Answer:</b><br/>
...

In [12]:
# code here

# Data Quality
<b>15 points</b>

<i>
<b>Description:</b><br/>
Verify data quality: Explain any missing values, duplicate data, and outliers. Are those mistakes? How do you deal with these problems? Give justifications for your methods.
</i>

<b>Answer:</b><br/>
...

In [13]:
# View info
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
parcelid                        int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottub

In [14]:
# View preliminary statistics
df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,811519.000000,6061.000000,1628.000000,2.973755e+06,2.973767e+06,12629.000000,1.938488e+06,2.856305e+06,17096.0,...,2647.000000,2.925289e+06,682069.000000,2.930235e+06,2.942667e+06,2.973778e+06,2.917484e+06,2.953967e+06,56464.000000,2.910091e+06
mean,1.332586e+07,1.931166,7.202607,646.883292,2.209143e+00,3.088949e+00,3.725948,5.784787e+00,2.299263e+00,66.0,...,278.296562,1.964262e+03,1.401464,1.708836e+05,4.204790e+05,2.014999e+03,2.524780e+05,5.377607e+03,13.892409,6.048431e+13
std,7.909966e+06,3.148587,2.436290,538.793473,1.077754e+00,1.275859e+00,0.501700,1.805352e+00,1.000736e+00,0.0,...,369.731508,2.344132e+01,0.539076,4.020683e+05,7.263467e+05,3.683161e-02,4.450132e+05,9.183107e+03,2.581006,3.249035e+11
min,1.071172e+07,1.000000,2.000000,20.000000,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,66.0,...,10.000000,1.801000e+03,1.000000,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,1.340000e+00,0.000000,-1.000000e+00
25%,1.164371e+07,1.000000,7.000000,272.000000,2.000000e+00,2.000000e+00,3.000000,4.000000e+00,2.000000e+00,66.0,...,96.000000,1.950000e+03,1.000000,7.480000e+04,1.796750e+05,2.015000e+03,7.483600e+04,2.461070e+03,14.000000,6.037400e+13
50%,1.254509e+07,1.000000,7.000000,534.000000,2.000000e+00,3.000000e+00,4.000000,7.000000e+00,2.000000e+00,66.0,...,168.000000,1.963000e+03,1.000000,1.225900e+05,3.060860e+05,2.015000e+03,1.670420e+05,3.991780e+03,14.000000,6.037572e+13
75%,1.409712e+07,1.000000,7.000000,847.250000,3.000000e+00,4.000000e+00,4.000000,7.000000e+00,3.000000e+00,66.0,...,320.000000,1.981000e+03,2.000000,1.968890e+05,4.880000e+05,2.015000e+03,3.069180e+05,6.201005e+03,15.000000,6.059042e+13
max,1.696019e+08,13.000000,27.000000,8516.000000,2.000000e+01,2.000000e+01,5.000000,1.200000e+01,2.000000e+01,66.0,...,6141.000000,2.015000e+03,41.000000,2.514860e+08,2.827860e+08,2.016000e+03,9.024622e+07,3.458861e+06,99.000000,4.830301e+14


### Missing Values

Several missing values were found which show up as NaN in the dataframe. 

In [15]:
# Impute NaN ordinal values with 0
df[['parcelid','airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid',
                    'buildingqualitytypeid','calculatedbathnbr','decktypeid','fips','fireplacecnt',
                    'fullbathcnt','garagecarcnt','garagetotalsqft','hashottuborspa','heatingorsystemtypeid',
                    'latitude','longitude','poolcnt','pooltypeid10','pooltypeid2','pooltypeid7',
                    'propertylandusetypeid','rawcensustractandblock','regionidcity','regionidcounty',
                    'regionidneighborhood','regionidzip','roomcnt','storytypeid','threequarterbathnbr',
                    'typeconstructiontypeid','unitcnt','yardbuildingsqft17','yearbuilt','numberofstories',
                    'fireplaceflag','structuretaxvaluedollarcnt','taxvaluedollarcnt','assessmentyear',
                    'landtaxvaluedollarcnt','taxamount','taxdelinquencyyear','censustractandblock']] = df[['parcelid','airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid',
                    'buildingqualitytypeid','calculatedbathnbr','decktypeid','fips','fireplacecnt',
                    'fullbathcnt','garagecarcnt','garagetotalsqft','hashottuborspa','heatingorsystemtypeid',
                    'latitude','longitude','poolcnt','pooltypeid10','pooltypeid2','pooltypeid7',
                    'propertylandusetypeid','rawcensustractandblock','regionidcity','regionidcounty',
                    'regionidneighborhood','regionidzip','roomcnt','storytypeid','threequarterbathnbr',
                    'typeconstructiontypeid','unitcnt','yardbuildingsqft17','yearbuilt','numberofstories',
                    'fireplaceflag','structuretaxvaluedollarcnt','taxvaluedollarcnt','assessmentyear',
                    'landtaxvaluedollarcnt','taxamount','taxdelinquencyyear','censustractandblock']].fillna(value=0)
# df_imputed = df.fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
parcelid                        int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottub

### Modify Data Types

Once missing values were imputed with zeros, data types can be set appropriately.

In [26]:
# replace missing values with -1
df = df.replace(to_replace='?',value=-1)

# format continuous variables
continuous_features = ['basementsqft','bathroomcnt','bedroomcnt','finishedfloor1squarefeet',
                       'calculatedfinishedsquarefeet','finishedsquarefeet12','finishedsquarefeet13',
                       'finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','lotsizesquarefeet',
                       'poolsizesum']

# format ordinal/integer variables
# includes integers that are categorical
ordinal_features = ['parcelid','airconditioningtypeid','architecturalstyletypeid','buildingclasstypeid',
                    'buildingqualitytypeid','calculatedbathnbr','decktypeid','fips','fireplacecnt',
                    'fullbathcnt','garagecarcnt','garagetotalsqft','hashottuborspa','heatingorsystemtypeid',
                    'latitude','longitude','poolcnt','pooltypeid10','pooltypeid2','pooltypeid7',
                    'propertylandusetypeid','rawcensustractandblock','regionidcity','regionidcounty',
                    'regionidneighborhood','regionidzip','roomcnt','storytypeid','threequarterbathnbr',
                    'typeconstructiontypeid','unitcnt','yardbuildingsqft17','yearbuilt','numberofstories',
                    'fireplaceflag','structuretaxvaluedollarcnt','taxvaluedollarcnt','assessmentyear',
                    'landtaxvaluedollarcnt','taxamount','taxdelinquencyyear','censustractandblock']

# format string variables
string_features = ['propertycountylandusecode','propertyzoningdesc','taxdelinquencyflag']

#df[continuous_features] = [continuous_features].astype(np.float64)
df[ordinal_features] = df[ordinal_features].astype(np.int64)
#df[string_features] = df[string_features].astype(str)

# After imputation and modifiying data types, view of new data info and preliminary statistics
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
parcelid                        int64
airconditioningtypeid           int64
architecturalstyletypeid        int64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             int64
buildingqualitytypeid           int64
calculatedbathnbr               int64
decktypeid                      int64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            int64
fireplacecnt                    int64
fullbathcnt                     int64
garagecarcnt                    int64
garagetotalsqft                 int64
hashottuborspa                 

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,2.985217e+06,2.985217e+06,1628.000000,2.973755e+06,2.973767e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,...,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.332586e+07,5.249796e-01,1.462373e-02,646.883292,2.209143e+00,3.088949e+00,1.576267e-02,3.756424e+00,2.148178e+00,3.779745e-01,...,1.924829e+03,3.202096e-01,1.729523e-03,1.677362e+05,4.144857e+05,2.007278e+03,2.467494e+05,5.320825e+03,2.627685e-01,5.896216e+13
std,7.909966e+06,1.852887e+00,3.422926e-01,538.793473,1.077754e+00,1.275859e+00,2.440230e-01,3.120234e+00,1.073639e+00,4.980307e+00,...,2.764763e+02,6.423593e-01,4.155156e-02,3.990106e+05,7.228719e+05,1.244937e+02,4.415392e+05,9.151294e+03,1.925471e+00,9.479036e+12
min,1.071172e+07,0.000000e+00,0.000000e+00,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,1.164371e+07,0.000000e+00,0.000000e+00,272.000000,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,...,1.950000e+03,0.000000e+00,0.000000e+00,7.216500e+04,1.738980e+05,2.015000e+03,6.821400e+04,2.409000e+03,0.000000e+00,6.037301e+13
50%,1.254509e+07,0.000000e+00,0.000000e+00,534.000000,2.000000e+00,3.000000e+00,0.000000e+00,4.000000e+00,2.000000e+00,0.000000e+00,...,1.963000e+03,0.000000e+00,0.000000e+00,1.205040e+05,3.018630e+05,2.015000e+03,1.624370e+05,3.956000e+03,0.000000e+00,6.037555e+13
75%,1.409712e+07,1.000000e+00,0.000000e+00,847.250000,3.000000e+00,4.000000e+00,0.000000e+00,7.000000e+00,3.000000e+00,0.000000e+00,...,1.981000e+03,0.000000e+00,0.000000e+00,1.947780e+05,4.840000e+05,2.015000e+03,3.022000e+05,6.166000e+03,0.000000e+00,6.059042e+13
max,1.696019e+08,1.300000e+01,2.700000e+01,8516.000000,2.000000e+01,2.000000e+01,5.000000e+00,1.200000e+01,2.000000e+01,6.600000e+01,...,2.015000e+03,4.100000e+01,1.000000e+00,2.514860e+08,2.827860e+08,2.016000e+03,9.024622e+07,3.458861e+06,9.900000e+01,4.830301e+14


# Simple Statistics
<b>10 points</b>

<i>
<b>Description:</b><br/>
Visualize appropriate statistics (e.g., range, mode, mean, median, variance, counts) for a subset of attributes. Describe anything meaningful you found from this or if you found something potentially interesting. Note: You can also use data from other sources for comparison. Explain why the statistics run are meaningful. 
</i>

<b>Answer:</b><br/>
...

In [16]:
# code here

# Visualize Attributes
<b>15 points</b>

<i>
<b>Description:</b><br/>
Visualize the most interesting attributes (at least 5 attributes, your opinion on what is interesting). Important: Interpret the implications for each visualization. Explain for each attribute why the chosen visualization is appropriate.
</i>

<b>Answer:</b><br/>
...

In [17]:
# code here

# Explore Joint Attributes
<b>15 points</b>

<i>
<b>Description:</b><br/>
Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.
</i>

<b>Answer:</b><br/>
...

In [18]:
# code here

# Explore Attributes and Class
<b>10 points</b>

<i>
<b>Description:</b><br/>
Identify and explain interesting relationships between features and the class you are trying to predict (i.e., relationships with variables and the target classification).
</i>

<b>Answer:</b><br/>
...

In [19]:
# code here

# New Features
<b>5 points</b>

<i>
<b>Description:</b><br/>
Are there other features that could be added to the data or created from existing features? Which ones?
</i>

<b>Answer:</b><br/>
...

In [20]:
# code here

# Exceptional Work
<b>10 points</b>

<i>
<b>Description:</b><br/>
You have free reign to provide additional analyses. One idea: implement dimensionality reduction, then visualize and interpret the results.
</i>

<b>Answer:</b><br/>
...

In [21]:
# code here